# 三步教你用BERT
问题：这才第二篇就开始BERT了？？

是的，有一说一，会读写数据之后确实可以用BERT。因为BERT该做的都帮你做好了。。。

接下来先简单夸一下BERT：

BERT可以搞定很多nlp任务，包括推理、情感分析、问答。虽然距离BERT提出已经快两年了，但目前很多项目仍旧会先用它跑，大部分任务用BERT不优化的情况下精度都能上90%。没错BERT就是如此强大！尽管目前有性能更好的XL-Net、SemBERT等等，依旧无法抵挡BERT的存在。

接下来我们来下载谷歌官方的原版[BERT](https://github.com/google-research/bert)。

然后你会看到很多关于BERT模型的介绍，感兴趣可以戳[原文](https://arxiv.org/pdf/1810.04805.pdf)。

下面以我浅薄的认识给大家概括一下BERT是啥。

BERT分为两部分，pre-train和fine-tune。pre-train就是先用大量的语料训练模型，得到预训练的embedding。fine-tune就是以预训练为基础，在下游任务上再次训练embedding。一般我们用BERT都是用它预训练好的模型，在上面跑fine-tune。所以这篇的完整题目应该叫做：教你基于BERT预训练来fine-tune。

BERT预训练部分有两个任务：
* 完形填空：简单来说就是把文章里一些词用特殊标记覆盖掉，然后让模型预测这个地方的词
* 预测下一句：由于要适应推理方面的任务，光训练词还不够，因此BERT预训练的时候还需要根据前一句话去预测下一句

整个BERT都基于名为[Transformer](https://arxiv.org/pdf/1706.03762.pdf)的编解码模型，整个模型都采用self-attention，达到并行编码的目的。（问题：解码能不能并行？等我再去看看源码？）

好了终于扯完了，接下里开始教你用BERT来fine-tune！

三步：
* 配置环境，下载预训练模型
* 在run_classifier.py添加任务进程
* 跑模型

## 第一步
首先你要启一个Tensorflow的环境，cpu或者gpu都行，但要保证环境里面tensorflow-xxx的版本都一致，否则会报错。

下载预训练模型，比如[中文](https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip)。

里面模型还挺多的，练手的话可以先用base，large的模型很大，cased/uncased是大小写区不区分，如果你是中文的话就只有一个选择。

下载好之后解压，你会看一个`.ckpt`文件，这个就是我们要用的模型。另外vocab是词表，可以打开看一下里面都有哪些语料，包括一些特殊标记。然后把`chinese_L-12_H-768_A-12`这个文件夹放到全是代码的那个目录里。

## 第二步
这步稍微复杂点，需要修改代码。

首先你需要知道BERT能跑哪些任务，虽然实际上它都能跑。。。只要你会用。这里先介绍一下最简单的sentence/sentence-pair的分类任务，也是谷歌网页上给出的。顾名思义就是一句话或者两句话，多句话你可以把它们拼接起来，随后有个分类标签，也就是：
* text_a
* text_b（可无）
* label

知道这个之后，我们打开`run_classifier.py`，找到第783行。

In [ ]:
def main(_):
    tf.logging.set_verbosity(tf.logging.INFO)

    processors = {
      "cola": ColaProcessor,
      "mnli": MnliProcessor,
      "mrpc": MrpcProcessor,
      "xnli": XnliProcessor,
    }

这里有个processors，这四个是它默认可选的任务，乍一看确实是句子对类型的任务。

我们在里面添加一个自己的进程。

In [ ]:
def main(_):
    tf.logging.set_verbosity(tf.logging.INFO)

    processors = {
      "cola": ColaProcessor,
      "mnli": MnliProcessor,
      "mrpc": MrpcProcessor,
      "xnli": XnliProcessor,
      "xxxx": XxxxProcessor,
    }

添加完名字之后，你得告诉BERT这个具体是什么任务，代码网上翻，找到177行。

这里开始有很多个class，可以发现最上面的DataProcessor是基类，下面的类都是它的实现，且名字与刚刚的processors完全对应。

所以我们也要写一个类似的继承类。（其实很简单，因为这个类的作用就是告诉BERT我们的数据长什么样）

首先我们先随便复制一个其中的类，名字先改成XxxxProcessor，这里复制的是MnliProcessor。

In [2]:
class XxxxProcessor(DataProcessor):
    """Processor for the MultiNLI data set (GLUE version)."""

    def get_train_examples(self, data_dir): 
        """See base class."""
        return self._create_examples(
          self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
          self._read_tsv(os.path.join(data_dir, "dev_matched.tsv")), "dev_matched")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
          self._read_tsv(os.path.join(data_dir, "test_matched.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, tokenization.convert_to_unicode(line[0]))
            text_a = tokenization.convert_to_unicode(line[8])
            text_b = tokenization.convert_to_unicode(line[9])
            if set_type == "test":
                label = "contradiction"
            else:
                label = tokenization.convert_to_unicode(line[-1])
            examples.append(
              InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

然后可以看到前三个函数是分别获取训练集、验证集和测试集，并且默认格式是tsv。如果你想用csv或者别的当然也可以，那就修改一下前面的read_tsv函数，在197行。

In [ ]:
def _read_tsv(cls, input_file, quotechar=None):
    """Reads a tab separated value file."""
    with tf.gfile.Open(input_file, "r") as f:
        reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
        lines = []
        for line in reader:
            lines.append(line)
        return lines

可以发现这个函数其实很简单，结合上一篇的内容（最基础部分），它就是把tsv文件的每一行加入到lines数组里。

注意：这里是tsv，如果你想用csv就改一下reader的参数，甚至你想用json都可以改。这里为了方便就用tsv，代码可以完全不用改。你只要把你的数据集都变成tsv格式，名字和代码里对应就行，这一步可以先放一放。

接下来看到get_labels函数，就是告诉模型你的标签有几种。现在假设我们的任务是一个简单二分类，标签0和1，标签是文字也是可以的，但必须是字符串。

In [ ]:
def get_labels(self):
    """See base class."""
    return ["0", "1"]

最后就是create_examples函数，可以发现参数里的lines就是read_tsv返回的lines。i=0的时候continue是因为第一行是表头。

接下来就是根据你的数据集来写了，假设我们的tsv文件长这样：
```tsv
A B Label
I am a boy. I am not a girl. 0
I am 60 years old. I am a boy. 1
```
那么我们的text_a就是line\[0\], text_b就是line\[1\]，label就是line\[-1\]。guid是id，可以不管。

In [ ]:
def _create_examples(self, lines, set_type):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, line) in enumerate(lines):
        if i == 0:
            continue
        guid = "%s-%s" % (set_type, tokenization.convert_to_unicode(line[-1]))
        text_a = tokenization.convert_to_unicode(line[0])
        text_b = tokenization.convert_to_unicode(line[1])
        label = tokenization.convert_to_unicode(line[-1])
        examples.append(
            InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    return examples

没错，就是这样！

最后我们的训练样本会通过IntputExample送到模型里。

代码就这样修改完了，其实整个添加的过程很随意，text_a和text_b根据你实际的数据集来就行。如果你代码玩得溜，你可以随便怎么改。

## 第三步
**训练**
```bash
export BERT_BASE_DIR=chinese_L-12_H-768_A-12
export GLUE_DIR=data
export OUTPUT_DIR=output
```
BERT_BASE_DIR是告诉BERT预训练文件在哪，之前已经放到同一个目录里。GLUE_DIR是数据集的路径，需要自己新建。我这里新建了一个`data`文件夹，记得把你自己的`train.tsv`/`dev_matched.tsv`/`test_matched.tsv`三个文件放进去。最好再指定一个输出路径OUTPUT_DIR存放结果，后面还会用到。

然后就能训练了。
```bash
python run_classifier.py \
  --task_name=XXXX \
  --do_train=true \
  --do_eval=true \
  --data_dir=$GLUE_DIR/ \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --max_seq_length=32 \
  --train_batch_size=128 \
  --learning_rate=2e-5 \
  --num_train_epochs=3.0 \
  --output_dir=$OUTPUT_DIR/
  ```
 task_name注意改成自己的名字，如果你的内存比较小可以减小train_batch_size，如果你的句子比较短也可以减小max_seq_length。
 
 **预测**
 
2000 years later...

训练好的模型会存在output里，同样是ckpt文件，另外还能找到一个eval的文件，里面会告诉你验证集的精度和损失等等。

这里当然要用我们刚跑出来的模型作为分类器。
```bash
export TRAINED_CLASSIFIER=output
```
开始预测！这里同样输出到output里。
```bash
python run_classifier.py \
  --task_name=XXXX \
  --do_predict=true \
  --data_dir=$GLUE_DIR/ \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$TRAINED_CLASSIFIER \
  --max_seq_length=128 \
  --output_dir=$OUTPUT_DIR/
 ```
 **结果**
 
 预测很快就能跑完，结果在output里一个叫`test_results.tsv`的文件。打开之后是一群浮点数，分别对应每个标签的概率。比如我的输出应该长这样：
 ```tsv
 0.9432 0.0568
 0.0463 0.9537
 ```
 你可以简单写个程序把它们转换成你要的标签~

In [ ]:
def toLabel(file_name):
    labels = []
    with open(file_name, 'r', encoding='utf-8') as f:
        file = csv.reader(f, delimiter='\t')
        for line in file:
            label = 0 if line[0] > line[1] else 1
            labels.append(label)
    return labels

三步跑BERT就这样搞定了，是不是很快！

不过BERT并没有这么简单，还有很多用法，比如BERT+xxx、xxxBERT。这里只是入门级的用法，要想用得好得研究一下源码和原文。

感谢收看！

## 下回：NLP前的数据预处理